In [29]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

myseed = 5  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [30]:
from tensorboardX import SummaryWriter 
#SummaryWriter 是一個類別，包含這套件的所有功能。

writer = SummaryWriter(comment='test')
#在預設資料夾後面加上註解 檔名變為：'runs/Aug20-obov01-3xLR'

In [31]:
if torch.cuda.is_available():
  print("is available")

is available


In [32]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.n1 = nn.Linear(2,1,bias=False)
    self.n2 = nn.Linear(2,1,bias=False)
    self.criterion = nn.MSELoss(reduction='mean')
  def forward(self, x):
    output = (self.n1(x)*self.n2(x)).squeeze(1)
    return output

  def cal_loss(self, pred, target):
    ''' Calculate loss '''
    # TODO: you may implement L1/L2 regularization here
    
    return self.criterion(pred, target)
  

In [33]:
def train(tr_set, dv_set, model, config, device):
    ''' DNN training '''

    n_epochs = config['n_epochs']  # Maximum number of epochs

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    iter_idx = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record['train'].append(mse_loss.detach().cpu().item())
            writer.add_scalar('myscalar', mse_loss.detach().cpu().item(), iter_idx)
            iter_idx+=1
        # After each epoch, test your model on the validation (development) set.
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            # Save model if your model improved
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'
                .format(epoch + 1, min_mse))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

In [34]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
        
    
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss



In [35]:


def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = True                    # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 150,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        # 'lr': 0.001,                 # learning rate of SGD
        # 'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth'  # your model will be saved here
}

In [36]:
model = Model().to(device)  # Construct model and move to device

In [37]:
class RandomDataset(Dataset):
    ''' Dataset for loading and preprocessing the COVID19 dataset '''
    def __init__(self,
                 data,
                 mode='train',
                 target_only=False):
        self.mode = mode

        target = data[1]
        data = data[0]
        
        # Splitting training data into train & dev sets
        if mode == 'train':
            indices = [i for i in range(len(data)) if i % 10 != 0]
        elif mode == 'dev':
            indices = [i for i in range(len(data)) if i % 10 == 0]
        
        # Convert data into PyTorch tensors
        self.data = data[indices].type(torch.FloatTensor)
        # self.data = torch.FloatTensor(data[indices])
        self.target = target[indices].type(torch.FloatTensor)
        # self.target = torch.FloatTensor(target[indices])
        
        self.dim = self.data.shape[1]

        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
        # Returns the size of the dataset
        return len(self.data)

In [38]:
# dataset = RandomDataset((tr_x,tr_y), 'train', target_only=False)
# a,b=dataset[0]
# print(a,b)

In [39]:
def prep_dataloader(data, mode, batch_size, n_jobs=0, target_only=False):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = RandomDataset(data, mode=mode, target_only=target_only)  # Construct dataset
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'), drop_last=False,
        num_workers=n_jobs, pin_memory=True)                            # Construct dataloader
    return dataloader

In [40]:
tr_x = torch.randint(100,(3000,2))
tr_y = tr_x[:,0]*tr_x[:,1]




tr_set = prep_dataloader((tr_x,tr_y), 'train', config['batch_size'], target_only=target_only)
dv_set = prep_dataloader((tr_x,tr_y), 'dev', config['batch_size'], target_only=target_only)



Finished reading the train set of COVID19 Dataset (2700 samples found, each dim = 2)
Finished reading the dev set of COVID19 Dataset (300 samples found, each dim = 2)


In [41]:
# for x in tr_set:
#   print(x[0])

In [42]:
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

Saving model (epoch =  133, loss = 987.2016)
Saving model (epoch =  134, loss = 936.7060)
Saving model (epoch =  135, loss = 888.7576)
Saving model (epoch =  136, loss = 842.6972)
Saving model (epoch =  137, loss = 796.6219)
Saving model (epoch =  138, loss = 755.0811)
Saving model (epoch =  139, loss = 713.9149)
Saving model (epoch =  140, loss = 674.8592)
Saving model (epoch =  141, loss = 636.5764)
Saving model (epoch =  142, loss = 601.2805)
Saving model (epoch =  143, loss = 567.2397)
Saving model (epoch =  144, loss = 534.5833)
Saving model (epoch =  145, loss = 503.5683)
Saving model (epoch =  146, loss = 473.8981)
Saving model (epoch =  147, loss = 446.0073)
Saving model (epoch =  148, loss = 419.5092)
Saving model (epoch =  149, loss = 393.8761)
Saving model (epoch =  150, loss = 369.7940)
Saving model (epoch =  151, loss = 346.8821)
Saving model (epoch =  152, loss = 325.2187)
Saving model (epoch =  153, loss = 304.8078)
Saving model (epoch =  154, loss = 285.1795)
Saving mod

In [43]:
for param in model.parameters():
  print(param.data)

tensor([[ 8.7371e-01, -6.7276e-07]], device='cuda:0')
tensor([[1.7575e-06, 1.1445e+00]], device='cuda:0')


In [16]:
m = Model(4)
m.load_state_dict(torch.load('models/model.pth'))
for param in m.parameters():
  print(param.data)

tensor([[1., 1., 1., 1.]])
